In [3]:
# setting up the environment
import os
import re
# from os.path import join, isfile, splitext
# from os import listdir
# from IPython.core.display import display, HTML
import sys
import yaml

from nomorwhat import *


%load_ext autoreload
%autoreload 2

macro_config_file = 'macros.yml'
# input_file = 'test_input.txt'
input_file = 'test_100_input.txt'
input_file = 'test_500_input.txt'
input_file = 'test_1000_input.txt'
# output_file = 'test_output.txt'
output_file = 'test_100_output.txt'
output_file = 'test_500_output.txt'
output_file = 'test_1000_output.txt'
# orig_annot_file = 'test_full_annot.txt'
orig_annot_file = 'test_100_full_annot.txt'
orig_annot_file = 'test_500_full_annot.txt'
orig_annot_file = 'test_1000_full_annot.txt'
# merged_output_file = 'test_merged_output.txt'
merged_output_file = 'test_100_merged_output.txt'
merged_output_file = 'test_500_merged_output.txt'
merged_output_file = 'test_1000_merged_output.txt'


In [3]:
# reading macros
with open(macro_config_file, 'r') as fin:
    macros = yaml.load(fin)
# my_sentences = readtext(input_file)

/home/ligeti/bin/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
def check_macro(macro_name, anal):
    macro_res = False
    # print(macro_name, anal)
    if macros['macros'][macro_name]['type'] == 'list':
        if anal[0] in macros['macros'][macro_name]['value']:
            macro_res = True
        else:
            macro_res = False
    elif macros['macros'][macro_name]['type'] == 'regex':
        if macros['macros'][macro_name]['regexp_type'] == 'search':
            macro_res = re.search(macros['macros'][macro_name]['value'], anal[2])
        else:
            pass
    elif macros['macros'][macro_name]['type'] == 'ends':
        macro_res = anal[2].endswith(macros['macros'][macro_name]['value'])
    elif macros['macros'][macro_name]['type'] == 'complex':
        if macros['macros'][macro_name]['compl_type'] == 'and': 
            all_true = True
            for macro in macros['macros'][macro_name]['sub_macros']:
                if not check_macro(macro,anal):
                    all_true = False
                    break
            macro_res = all_true
        elif macros['macros'][macro_name]['compl_type'] == 'or':
            one_true = False
            for macro in macros['macros'][macro_name]['sub_macros']:
                if check_macro(macro,anal):
                    one_true = True
                    break
            macro_res = one_true
    elif macros['macros'][macro_name]['type'] == 'neg':
        macro_res = not check_macro(macros['macros'][macro_name]['sub_macro'],anal)
            
    return macro_res


In [5]:
#macro_name = 'nu_mn'
#macros['macros'][macro_name]

if check_macro('not_gen', ['ez', 'ez', 'FN_NM.NOM']):
    print("ugyesvagyez")
if check_macro('NPMod', ['szép', 'szép', 'MN.NOM']):
    print("ugyesvagy")
if check_macro('NOM', ["almák", 'alma', 'FN.PL.NOM']):
    print("ugyesvagy")
if check_macro('not_kop_v', ['válaszolt', 'válaszol', 'IGE.Me3']):
    print("ugyesvagyte")
if check_macro('def_art', ['a', 'a', 'DET']):
    print('ugyesvagy')
if not check_macro('def_art', ['az', 'az', 'FN_NM.NOM']):
    print('ugyesvagy')
if check_macro('full_stop', ['.', '.', 'SPUNCT']):
    print('ugyI')
if not check_macro('Adj_tree', ['szépek', 'szép', 'MN.PL.NOM']):
    print('ugyesvagy')

if check_macro('NPMod', ['1848_49-es', '1848_49-es', 'MN.NOM']):
    print("na")

ugyesvagyez
ugyesvagy
ugyesvagy
ugyesvagyte
ugyesvagy
ugyesvagy
ugyI
ugyesvagy
na


In [6]:
def NUM_rules(window, curr_POS):
    first_right_word, first_right_lemma, first_right_annot = window[0]
    second_right_word, second_right_lemma, second_right_annot = window[1]
    
    if re.search("FN|SZN|MN|NU", first_right_annot): # if the next token is a nominal, or a postposition, the word gets a "none"
        curr_POS = curr_POS.replace('NOM', 'none')
        
    elif check_macro('not_kop_v', window[0]): # if the next token is verb, but not a copula, the word is a nominative
        curr_POS = curr_POS.replace('NOM', 'nom')
        
    elif check_macro('def_art', window[0]): # or re.search('HA', first_right_annot): # if the next token is a definite article or an adverb, the word is a nominative
        curr_POS = curr_POS.replace('NOM', 'nom')

        
    else:
        curr_POS = curr_POS.replace('NOM', 'defnone')
        if check_macro('PUNCT', window[1]) or check_macro('def_art', window[1]) or re.search('HA', second_right_annot) or check_macro('V', window[1]):
            curr_POS = curr_POS.replace('defnone', 'none')
        
    return curr_POS

    

In [7]:
def ADJ_rules(window, curr_POS):
    first_right_word, first_right_lemma, first_right_annot = window[0]
    # print(window)
    second_right_word, second_right_lemma, second_right_annot = window[1]
    
    if check_macro('NUs', window[0]): # if the next token is postposition(al like element), the word gets a 'none'
        curr_POS = curr_POS.replace('NOM', 'none')
    
    elif check_macro('not_kop_v', window[0]): # if the next token is a verb, but not a copula, the word is a nominative
        curr_POS = curr_POS.replace('NOM', 'nom')
    
    elif check_macro('szn', window[0]): # before a numeral: default value
        curr_POS = curr_POS.replace('NOM', 'suff')
        
        if check_macro('PSE', window[1]): # if the second token has a poss. suff.
            curr_POS = curr_POS.replace('suff', 'gen')
        else: # otherwise
            curr_POS = curr_POS.replace('suff', 'nom')
    
    elif check_macro('full_stop', window[0]): # if the next token is a fix outsider, this word must be the end of an NP, thus 'nom'
        curr_POS = curr_POS.replace('NOM', 'nom')
    
    else: # otherwise
        curr_POS = curr_POS.replace('NOM', 'defnone')
        if check_macro('full_stop', window[1]) or check_macro('V', window[1]):# or check_macro('esetragos', window[1]): # if there is a punct.mark in the window
            curr_POS = curr_POS.replace('defnone', 'none')
            
    return curr_POS    

In [16]:
def NOUN_rules(window, curr_POS, curr_word, token):
    first_right_word, first_right_lemma, first_right_annot = window[0]
    second_right_word, second_right_lemma, second_right_annot = window[1]
    if check_macro('NUs', window[0]): # if the next token is postposition(al like element), the word gets a 'none'
        curr_POS = curr_POS.replace('NOM', 'none')
        
    elif check_macro('not_gen', token):
        curr_POS = curr_POS.replace('NOM', 'nom')
        # print('notgen')
    
    elif check_macro('PSE', window[0]):
        curr_POS = curr_POS.replace('NOM', 'gen')
        
    elif check_macro('not_kop_v', window[0]) or check_macro('pl', window[0]):# or first_right_annot == 'IK':
        curr_POS = curr_POS.replace('NOM', 'nom')
        # print('notkop')
    elif check_macro('TULN', window[0]):
        curr_POS = curr_POS.replace('NOM', 'suff')
        
        if not check_macro('cimunevu', window[1]):
            curr_POS = curr_POS.replace('suff', 'nom')
            # print('cimunevu')
    elif check_macro('NPMod', window[0]):
        curr_POS = curr_POS.replace('NOM', 'suff')
        # print("hurra")
        if check_macro('PSE', window[1]) and not re.search('PSe3', curr_POS):
            curr_POS = curr_POS.replace('suff', 'gen')
        elif check_macro('V', window[1]) or check_macro('PUNCT', window[1]) or check_macro('vonatk', window[1]) or re.search('PSe3', curr_POS):
            curr_POS = curr_POS.replace('suff', 'nom')       
            # print('V')
    elif check_macro('full_stop', window[0]) or first_right_word == ':' or first_right_word == '!':
        curr_POS = curr_POS.replace('NOM', 'nom')
        # print('fullstop')
    else:
        curr_POS = curr_POS.replace('NOM', 'suff')
        
        if check_macro('V', window[1]) or check_macro('PUNCT', window[1]) or check_macro('vonatk', window[1]) or re.search('PSe3', curr_POS) or second_right_word == "#":
            curr_POS = curr_POS.replace('suff', 'nom')       
            # print('masoikV')
        
    return curr_POS

In [8]:
def nom_or_what(s):
    sent = format_sents(s) 
    new_sent = [] # to store a sentence with the novel tags
    to_write_later = [] # to store the NOM token and the window to create the annotation file later
    for i in range(len(sent)):
        
        token = sent[i]
        curr_word = token[0]
        curr_lemma = token[1]
        curr_POS = token[2]
        
        if curr_word != "#": # if the given token is not the end of a sentence
        
            window = sent[i + 1:i + 3]

            
            if check_macro('NOM', token): # if the given token is a suffixless nominal
                
                if check_macro('Noun_tree', token): # if the given token is a noun, or a plural adjective, participle
                    curr_POS = NOUN_rules(window, curr_POS, curr_word, token)
                    # print(token)

                elif check_macro('Adj_tree', token): # if the given token is a singular adjective or participle
                    curr_POS = ADJ_rules(window, curr_POS)
        
                elif check_macro('Num_tree', token): # if the given token is a numeral
                    curr_POS = NUM_rules(window, curr_POS)
                
                new_token = curr_word + ' ' + curr_lemma + ' ' + curr_POS
                to_write_later.append( (window, new_token ) ) # a tuple of the window of the given token and the full (novel) annotation of the token
                
        new_sent.append(curr_word + '/' + curr_lemma + '/' + curr_POS)
    
    return (new_sent, to_write_later)


    # print(' '.join(new_sent))
    
    

In [9]:
def write_to_annot_file(new_sent, window, outp, i):
    outp.writelines(str(i) + '. ' + ' '.join([token.split('/')[0] for token in new_sent])+ '\n')
    [outp.writelines(('-' + nom.split()[0] + ' ' + window[0][0] + ' ' + window[1][0] + '\n', nom + '\n', nom + '\n', nom + '\n')) for (window, nom) in to_write]
    outp.writelines(('\n', '\n'))
    

In [19]:
# 1 sent, demo
new_sent, to_write = nom_or_what('a/a/DET 20./20./MN.NOM század/század/FN.NOM kiemelkedő/kiemelkedik/IK.IGE._OKEP.NOM irodalmi/irodalmi/MN.NOM egyéniségeit/egyéniség/FN.PSe3i.ACC is/is/HA ././SPUNCT')
with open ('egymondat', 'w') as outp:
    write_to_annot_file(new_sent,to_write, outp, 1)

    

        

In [21]:
# 10 sents, demo, file
with open(input_file, 'r') as inp, open(output_file, 'w') as outp:
    for i, line in enumerate(inp, start = 1):
        new_sent, to_write = nom_or_what(line)
        write_to_annot_file(new_sent, to_write, outp, i)
        


In [294]:
def merge_annot_with_outp():
    annot = open(orig_annot_file, 'r').readlines()
    outp = open(output_file, 'r').readlines()
    merged_outp = open(merged_output_file, 'w')
    print(len(annot))
    print(len(outp))
    j = -1
    for i in range(len(annot)):
        if i == j:
            merged_outp.write(outp[i])
        else:
            merged_outp.write(annot[i])
        if annot[i].startswith('-'):
            j = i+3
    merged_outp.close()
    
        

merge_annot_with_outp()

In [322]:
merge_annot_with_outp()

13286
13288


In [67]:
def evaluate_nom_or_what(annotated_file):
    manu_manu = {"nom_nom":0,"nom_suff":0,"suff_nom":0,"gen_gen":0,"gen_suff":0,
                 "suff_suff":0,"suff_gen":0,"none_none":0,"defnone_defnone":0,"defnone_none":0,"none_defnone":0, "other":0}
    manu_algo = {"nom_nom":0,"nom_suff":0,"suff_nom":0,"gen_gen":0,"gen_suff":0,
                 "suff_suff":0,"suff_gen":0,"none_none":0,"defnone_defnone":0,"defnone_none":0,"none_defnone":0, "other":0}
    full_algo = {"nom_nom":0,"nom_suff":0,"suff_nom":0,"gen_gen":0,"gen_suff":0,
                 "suff_suff":0,"suff_gen":0,"none_none":0,"defnone_defnone":0,"defnone_none":0,"none_defnone":0, "other":0}
    nom_nom = 0
    nom_suff = 0 
    suff_nom = 0
    gen_gen = 0
    gen_suff = 0 
    suff_suff = 0 
    suff_gen = 0
    none_none = 0
    defnone_defnone = 0
    defnone_none = 0
    none_defnone = 0
    xne = 0
    other = 0
    postag =0
    vok = 0    
    full_nom_nom = 0
    full_nom_suff = 0 
    full_suff_nom = 0
    full_gen_gen = 0
    full_gen_suff = 0 
    full_suff_suff = 0 
    full_suff_gen = 0
    full_none_none = 0
    full_defnone_defnone = 0
    full_defnone_none = 0
    full_none_defnone = 0
    full_other = 0
    full_nom_nom_algo = 0
    full_nom_suff_algo = 0 
    full_suff_nom_algo = 0
    full_gen_gen_algo = 0
    full_gen_suff_algo = 0 
    full_suff_suff_algo = 0 
    full_suff_gen_algo = 0
    full_none_none_algo = 0
    full_defnone_defnone_algo = 0
    full_defnone_none_algo = 0
    full_none_defnone_algo = 0
    full_other_algo = 0
    postag_full_algo = 0
    xne_full_algo = 0
    vok_full_algo = 0
    inf = open(annotated_file, 'r')
    for line in inf:
        if len(line)>2:
            if re.search("^[^0-9\-]", line.strip()):
                annot_manual_full_sent_line = line
                annot_manual_full_sent = line.split()[2]
                annot_manual_window_line = inf.readline()
                annot_manual_window = annot_manual_window_line.strip().split()[2]
                annot_automatic_line = inf.readline()
                annot_automatic = annot_automatic_line.strip().split()[2]
                if annot_manual_full_sent.endswith('postag_hiba'):
                    postag+=1
                elif annot_manual_full_sent.endswith("xne"):
                    xne+=1
                elif annot_manual_full_sent.endswith('vok'):
                    vok+= 1
                else:
                    # egyezés, teljes mondat - ablak, kézi:
                    if annot_manual_full_sent == annot_manual_window:
                        if annot_manual_full_sent.endswith("nom"):
                            manu_manu["nom_nom"]+=1
                        elif annot_manual_full_sent.endswith("gen"):
                            manu_manu["gen_gen"]+=1
                        elif annot_manual_full_sent.endswith("suff"):
                            manu_manu["suff_suff"]+=1
                        elif annot_manual_full_sent.endswith("defnone"):
                            manu_manu["defnone_defnone"]+=1
                        elif annot_manual_full_sent.endswith(".none"):
                            manu_manu["none_none"]+=1
                            
                    else:
                        # ha nem egyezik, nem anmk, nem vok, nem postag
                        if annot_manual_full_sent.endswith("nom") and annot_manual_window.endswith("suff"):
                            manu_manu["nom_suff"]+= 1
                        elif annot_manual_full_sent.endswith("gen") and annot_manual_window.endswith("suff"):
                            manu_manu["gen_suff"]+= 1
                        elif annot_manual_full_sent.endswith("suff") and annot_manual_window.endswith("nom"):
                            manu_manu["suff_nom"]+=1
                        elif annot_manual_full_sent.endswith("suff") and annot_manual_window.endswith("gen"):
                            manu_manu["suff_gen"]+=1
                        elif annot_manual_full_sent.endswith("defnone") and annot_manual_window.endswith(".none"):
                            manu_manu["defnone_none"]+=1
                        elif annot_manual_full_sent.endswith(".none") and annot_manual_window.endswith("defnone"):
                            manu_manu["none_defnone"]+=1
                        else:
                            manu_manu["other"]+=1
                            
                    # egyezés, teljes mondat - algo:
                    if annot_manual_full_sent == annot_automatic:
                        if annot_manual_full_sent.endswith("nom"):
                            full_algo["nom_nom"]+=1
                        elif annot_manual_full_sent.endswith("gen"):
                            full_algo["gen_gen"]+=1
                        elif annot_manual_full_sent.endswith("suff"):
                            full_algo["suff_suff"]+=1
                        elif annot_manual_full_sent.endswith("defnone"):
                            full_algo["defnone_defnone"]+=1
                        elif annot_manual_full_sent.endswith(".none"):
                            full_algo["none_none"]+=1
                    else:
                        # ha nem egyezik, nem anmk, nem vok, nem postag
                        if annot_manual_full_sent.endswith("nom") and annot_automatic.endswith("suff"):
                            full_algo["nom_suff"]+= 1
                        elif annot_manual_full_sent.endswith("gen") and annot_automatic.endswith("suff"):
                            full_algo["gen_suff"]+= 1
                        elif annot_manual_full_sent.endswith("suff") and annot_automatic.endswith("nom"):
                            full_algo["suff_nom"]+=1
                        elif annot_manual_full_sent.endswith("suff") and annot_automatic.endswith("gen"):
                            full_algo["suff_gen"]+=1
                        elif annot_manual_full_sent.endswith("defnone") and annot_automatic.endswith(".none"):
                            full_algo["defnone_none"]+=1
                        elif annot_manual_full_sent.endswith(".none") and annot_automatic.endswith("defnone"):
                            full_algo["none_defnone"]+=1
                        else:
                            full_algo["other"]+=1
                            
                    # egyezés, kézi ablak - algo:
                    if annot_manual_window == annot_automatic:
                        if annot_manual_window.endswith("nom"):
                            manu_algo["nom_nom"]+=1
                        elif annot_manual_window.endswith("gen"):
                            manu_algo["gen_gen"]+=1
                        elif annot_manual_window.endswith("suff"):
                            manu_algo["suff_suff"]+=1
                        elif annot_manual_window.endswith("defnone"):
                            manu_algo["defnone_defnone"]+=1
                        elif annot_manual_window.endswith(".none"):
                            manu_algo["none_none"]+=1
                    else:
                        # ha nem egyezik, nem anmk, nem vok, nem postag
                        if annot_manual_window.endswith("nom") and annot_automatic.endswith("suff"):
                            manu_algo["nom_suff"]+= 1
                        elif annot_manual_window.endswith("gen") and annot_automatic.endswith("suff"):
                            manu_algo["gen_suff"]+= 1
                        elif annot_manual_window.endswith("suff") and annot_automatic.endswith("nom"):
                            manu_algo["suff_nom"]+=1
                        elif annot_manual_window.endswith("suff") and annot_automatic.endswith("gen"):
                            manu_algo["suff_gen"]+=1
                        elif annot_manual_window.endswith("defnone") and annot_automatic.endswith(".none"):
                            manu_algo["defnone_none"]+=1
                        elif annot_manual_window.endswith(".none") and annot_automatic.endswith("defnone"):
                            manu_algo["none_defnone"]+=1
                        else:
                            manu_algo["other"]+=1
                            
    print("xne = " + str(xne))
    print("postag = " + str(postag))
    print("vok = "+ str(vok))
    print()
    print("Kézi-kézi eredmények:")
    print()
    for key in manu_manu:
        print(key,manu_manu[key])
    print()
    print()
    print("Kézi - algoritmus:")
    print()
    for key in manu_algo:
        print(key,manu_algo[key])
    print()
    print()
    print("Teljes mondat - algoritmus:")
    print()
    for key in full_algo:
        print(key,full_algo[key])
    print()

In [68]:
evaluate_nom_or_what("test_1000_merged_output.txt")

2551
amnk = 42
postag = 128
vok = 34

Kézi-kézi eredmények:

nom_nom 626
nom_nulla 187
nulla_nom 0
gen_gen 266
gen_nulla 54
nulla_nulla 17
nulla_gen 0
semmi_semmi 762
defsemmi_defsemmi 6
defsemmi_semmi 0
semmi_defsemmi 397
other 31

Counter a kézi-kéziben:
2346

Kézi - algoritmus:

nom_nom 621
nom_nulla 13
nulla_nom 19
gen_gen 257
gen_nulla 4
nulla_nulla 247
nulla_gen 4
semmi_semmi 623
defsemmi_defsemmi 364
defsemmi_semmi 42
semmi_defsemmi 131
other 21

Counter a kézi-algóban:
2346

Teljes mondat - algoritmus:

nom_nom 630
nom_nulla 181
nulla_nom 0
gen_gen 258
gen_nulla 54
nulla_nulla 16
nulla_gen 1
semmi_semmi 663
defsemmi_defsemmi 6
defsemmi_semmi 0
semmi_defsemmi 482
other 55
Counter a teljes mondat - algóban:
2346

